# Shubham Sharma
## IIT Bombay
This code is for predicting verbs using fusion network

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# For plotting, arrays and other things
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

#For Deep learning 
import keras
from keras import callbacks 
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.optimizers import RMSprop,SGD, adam #Optimizers
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
def fusion_network(width, height, depth, classes,weightsPath=None):
  
  bn_flag = True
  inputs = Input((width, height , depth))
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
  conv1 = BatchNormalization(axis = -1)(conv1,training=bn_flag)
  conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
  conv1 = BatchNormalization(axis= -1)(conv1,training=bn_flag)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  ##
  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
  conv2 = BatchNormalization(axis = -1)(conv2,training=bn_flag)
  conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
  conv2 = BatchNormalization(axis= -1)(conv2,training=bn_flag)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  ##
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
  conv3 = BatchNormalization(axis = -1)(conv3,training=bn_flag)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  ##
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
  conv4 = BatchNormalization(axis = -1)(conv4,training=bn_flag)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
  ##
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
  conv5 = BatchNormalization(axis = -1)(conv5,training=bn_flag)
  pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
  ##
  Y = Flatten()(pool5)
  Y = Dense(4096, activation='relu')(Y)
  Y = Dense(4096, activation='relu')(Y)
  Y = Dense(4096, activation='relu')(Y)
  out = Dense(classes, activation='softmax')(Y)
  
  model = Model(inputs=[inputs], outputs=[out])
  
  if weightsPath is not None:
    model.load_weights(weightsPath)
    
  print(model.summary())
  return model 
  
  

Loading and pre-processing the data

In [0]:
classes = 50 
path2load = 'gdrive/My Drive/Situation_recognition/data/'

X_name = np.load(path2load + 'X_name.npy')
Y_verbs = np.load(path2load + 'Y_verbs.npy')
dictionary_fifty_verbs = np.load(path2load + 'dictionary_fifty_verbs.npy')
X = np.load(path2load + 'X.npy')
Y = to_categorical(Y_verbs, classes)

# x_train = np.load(path2load + 'x_train.npy')
# x_test = np.load(path2load + 'x_test.npy')
# y_train = np.load(path2load + 'y_train.npy')
# y_test = np.load(path2load + 'y_test.npy')
# y_test = to_categorical(y_test, classes)
# y_train = to_categorical(y_train, classes)

In [0]:
path2save = 'gdrive/My Drive/Situation_recognition/fusion'
#Define Image Size and Number of classes
width, height, depth, classes = 256 , 256 , 3 , 50 
model = fusion_network(width, height, depth, classes , weightsPath='gdrive/My Drive/Situation_recognition/fusion')
print ("Compiling Model...")
#Store the network weights whenever loss is minimum than previous epoch
# modelCheck = callbacks.ModelCheckpoint(path2save+'_{epoch:04d}-{acc:.4f}.h5', monitor='loss', mode='auto')
modelCheck = callbacks.ModelCheckpoint(path2save, monitor='loss', verbose=0, mode='auto')
 
opt = adam(lr=1e-5)
#Set the compiler parameter for the training
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["categorical_accuracy"],sample_weight_mode='auto')
print ("Training the Model...")
 


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
batch_normalization_14 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_15 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 128, 128)     7385

In [0]:
#Train the Network
# history = model.fit(X, Y, batch_size = 4, epochs= 10, verbose=1, callbacks= [modelCheck], shuffle = True, validation_split = O.2)#,validation_data = (x_test , y_test)
history = model.fit(X , Y, batch_size=4, epochs=30, verbose=1, callbacks= [modelCheck], validation_split=0.2, shuffle=True)#,validation_data = (x_test , y_test)
print ("Dumping Weights to file...")

Train on 10703 samples, validate on 2676 samples
Epoch 1/30
10703/10703 [==============================] - 1376s 129ms/step - loss: 0.0803 - categorical_accuracy: 0.9820 - val_loss: 4.5530 - val_categorical_accuracy: 0.3188
Epoch 2/30
10703/10703 [==============================] - 1370s 128ms/step - loss: 0.0163 - categorical_accuracy: 0.9972 - val_loss: 5.0573 - val_categorical_accuracy: 0.3173
Epoch 3/30
  156/10703 [..............................] - ETA: 21:23 - loss: 3.1505e-04 - categorical_accuracy: 1.0000

KeyboardInterrupt: ignored

In [0]:
# path2save = 'gdrive/My Drive/Situation_recognition/fusion'
# #Define Image Size and Number of classes
# width, height, depth, classes = 256 , 256 , 3 , 50 
# model = fusion_network(width, height, depth, classes , weightsPath=None)
# print ("Compiling Model...")
# #Store the network weights whenever loss is minimum than previous epoch
# modelCheck = callbacks.ModelCheckpoint(path2save+'.h5', monitor='acc', mode='auto')
# # modelCheck = callbacks.ModelCheckpoint(path2save, monitor='acc', verbose=0, save_best_only=True, mode='auto')
 
# opt = adam(lr=1e-4)
# #Set the compiler parameter for the training
# model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["categorical_accuracy"],sample_weight_mode='auto')
# print ("Training the Model...")

# train_datagen = ImageDataGenerator(rescale=1./255)#, validation_split=0.2, interpolation_order=1, dtype='float32')
# # test_datagen = ImageDataGenerator(rescale=1./255)
# train_datagen.fit(X)
# # fits the model on batches with real-time data augmentation:
